Tiering of BloodHound data

Import libs

In [65]:
from neo4j import GraphDatabase
import pathlib

Create connection

In [66]:
class Neo4jConnection:
    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)
        
    def close(self):
        if self.__driver is not None:
            self.__driver.close()
        
    def query(self, query, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try: 
            session = self.__driver.session(database=db) if db is not None else self.__driver.session() 
            response = list(session.run(query))
        except Exception as e:
            print("Query failed:", e)
        finally: 
            if session is not None:
                session.close()
                
        return response
conn = Neo4jConnection(uri="bolt://localhost:7687", user="neo4j", pwd="bloodhound")
relpath = pathlib.Path().absolute().as_uri()

Tiering functions

In [83]:
tier_ous = '''
UNWIND value.ous as o
MATCH (obj) WHERE (obj.distinguishedname ENDS WITH o.path)
CALL apoc.create.addLabels(obj, ["Tier" + o.tier]) YIELD node
RETURN null
'''

tier_groups = '''
UNWIND value.groups as g
MATCH (group:Group) WHERE group.objectid ENDS WITH g.RID
CALL apoc.create.addLabels(group, ["Tier" + g.tier]) YIELD node
WITH group, g
MATCH (principal)-[:MemberOf*1..]->(group)
CALL apoc.create.addLabels(principal, ["Tier" + g.tier]) YIELD node
RETURN null
'''

tier_principals = '''
UNWIND value.principals as p
MATCH (principal) WHERE principal.objectid ENDS WITH p.RID
CALL apoc.create.addLabels(principal, ["Tier" + p.tier]) YIELD node
RETURN null
'''

Tier the data

In [84]:
# Generic tiering
query = '''
CALL apoc.load.json('%s/../jsondata/bloodhound/generic-tiering-bloodhound.json') YIELD value
%s
''' % (relpath, tier_ous)
out = conn.query(query)
query = '''
CALL apoc.load.json('%s/../jsondata/bloodhound/generic-tiering-bloodhound.json') YIELD value
%s
''' % (relpath, tier_groups)
out = conn.query(query)
query = '''
CALL apoc.load.json('%s/../jsondata/bloodhound/generic-tiering-bloodhound.json') YIELD value
%s
''' % (relpath, tier_principals)
out = conn.query(query)

# Customer specific tiering
query = '''
CALL apoc.load.json('%s/../jsondata/bloodhound/customer-tiering-bloodhound.json') YIELD value
%s
''' % (relpath, tier_ous)
out = conn.query(query)
query = '''
CALL apoc.load.json('%s/../jsondata/bloodhound/customer-tiering-bloodhound.json') YIELD value
%s
''' % (relpath, tier_groups)
out = conn.query(query)
query = '''
CALL apoc.load.json('%s/../jsondata/bloodhound/customer-tiering-bloodhound.json') YIELD value
%s
''' % (relpath, tier_principals)
out = conn.query(query)

# Add all nodes to the default tier
query = '''
CALL apoc.load.json('%s/../jsondata/bloodhound/generic-tiering-bloodhound.json') YIELD value
WITH value.defaulttier AS defaulttier
MATCH (obj)
CALL apoc.create.addLabels(obj, ["Tier" + defaulttier]) YIELD node
RETURN null
''' % relpath
out = conn.query(query)

# Delete higher tier labels for objects in multiple tiers
query = '''
MATCH (n) 
UNWIND labels(n) AS label
WITH n, label WHERE label STARTS WITH "Tier"
WITH n, label ORDER BY label ASC
WITH n, tail(collect(label)) AS wrongTiers
CALL apoc.create.removeLabels(n, wrongTiers)
YIELD node RETURN null
'''
out = conn.query(query)

# Set groups to be in the same tier as the highest tier of their members
query = '''
MATCH (group:Group)
MATCH (principal)-[:MemberOf*1..]->(group)
UNWIND labels(principal) AS allLabels
WITH DISTINCT allLabels, group WHERE allLabels STARTS WITH "Tier"
WITH group, allLabels ORDER BY allLabels DESC
WITH group, head(collect(allLabels)) AS rightTier
CALL apoc.create.setLabels(group, ["Base", "Group", rightTier]) YIELD node
RETURN null
'''
out = conn.query(query)

# Set OUs to be in the same tier as the lowest tier of their content
query = '''
MATCH (ou:OU)
WITH ou, size(ou.distinguishedname) AS oUSize
WITH ou, oUSize ORDER BY oUSize DESC
MATCH (ou)-[:Contains*1..]->(obj)
UNWIND labels(obj) AS allLabels
WITH DISTINCT allLabels, ou WHERE allLabels STARTS WITH "Tier"
WITH ou, allLabels ORDER BY allLabels ASC
WITH ou, head(collect(allLabels)) AS rightTier
CALL apoc.create.setLabels(ou, ["Base", "OU", rightTier]) YIELD node
RETURN null
'''
out = conn.query(query)

# Set Domains to be in Tier 0
query = '''
MATCH (domain:Domain)
CALL apoc.create.setLabels(domain, ["Base", "Domain", "Tier0"]) YIELD node
RETURN null
'''
out = conn.query(query)

# Set GPOs to be in the same tier as the lowest tier of the OUs (or domain) linked to
query = '''
MATCH (gpo:GPO)
MATCH (gpo)-[:GpLink]->(ou)
UNWIND labels(ou) AS allLabels
WITH DISTINCT allLabels, gpo WHERE allLabels STARTS WITH "Tier"
WITH gpo, allLabels ORDER BY allLabels ASC
WITH gpo, head(collect(allLabels)) AS rightTier
CALL apoc.create.setLabels(gpo, ["Base", "GPO", rightTier]) YIELD node
RETURN null
'''
out = conn.query(query)

Get names of objects in a given tier

In [56]:
query = '''
MATCH (obj:Tier0) RETURN obj.name LIMIT 10
'''
out = conn.query(query)

Get number of principals in a given tier

In [58]:
query = '''
MATCH (obj:Tier0) RETURN COUNT(obj)
'''
out = conn.query(query)

Get number of Tier 2 principals with path to Tier 0 principals

In [32]:
query = '''
MATCH (t0:Tier0)
CALL apoc.path.expandConfig(t0, {
    relationshipFilter: "<",
    labelFilter: ">Tier2",
    minLevel: 1,
    maxLevel: 3
})
YIELD path RETURN path
'''

Check if any node is more than one tier

In [63]:
query = '''
MATCH (n) 
UNWIND labels(n) AS allLabels
WITH DISTINCT allLabels, n WHERE allLabels STARTS WITH "Tier"
WITH n, size(collect(allLabels)) AS numOfTiers
WITH n WHERE numOfTiers > 1
RETURN n
'''
out = conn.query(query)
out

[]

Get number of objects in each tier

In [62]:
query = '''
CALL db.labels()
YIELD label WHERE label STARTS WITH "Tier"
WITH label ORDER BY label DESC
MATCH (n) WHERE label IN labels(n)
WITH label, count(n) as tierObjs
RETURN label, tierObjs
'''
out = conn.query(query)
out

[<Record label='Tier2' tierObjs=19236>,
 <Record label='Tier1' tierObjs=1466>,
 <Record label='Tier0' tierObjs=193>]

Check if number of tiered nodes match total number of nodes

In [64]:
query = '''
CALL db.labels()
YIELD label WHERE label STARTS WITH "Tier"
WITH label ORDER BY label DESC
MATCH (n) WHERE label IN labels(n)
WITH count(n) AS tierObjs
MATCH (n)
RETURN tierObjs, count(n)
'''
out = conn.query(query)
out

[<Record tierObjs=20895 count(n)=20895>]

Get Tier 2 -> Tier 0 relations

In [72]:
query = '''
MATCH path=(t2:Tier2)-[r]->(t0:Tier0) RETURN labels(t2), t2.name, TYPE(r), labels(t0), collect(t0.name) ORDER BY t2.name LIMIT 15
'''
out = conn.query(query)

Clean graph from Tier nodes and relations

In [82]:
query = '''
CALL db.labels()
YIELD label WHERE label STARTS WITH "Tier"
WITH collect(label) AS labels
MATCH (p)
WITH collect(p) AS people, labels
CALL apoc.create.removeLabels(people, labels)
YIELD node RETURN null
'''
out = conn.query(query)